<a href="https://colab.research.google.com/github/BenjaAR/lab13_aquino_reyes/blob/main/LAB13_AQUINO_REYES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd

# Subir archivo desde tu computadora
uploaded = files.upload()

# Leer el archivo CSV (asegúrate de que el nombre coincida exactamente)
df = pd.read_csv('garments_worker_productivity.csv')

# Mostrar las primeras filas del DataFrame
df.head()


Saving garments_worker_productivity.csv to garments_worker_productivity.csv


,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   day                    1197 non-null   object 
 4   team                   1197 non-null   int64  
 5   targeted_productivity  1197 non-null   float64
 6   smv                    1197 non-null   float64
 7   wip                    691 non-null    float64
 8   over_time              1197 non-null   int64  
 9   incentive              1197 non-null   int64  
 10  idle_time              1197 non-null   float64
 11  idle_men               1197 non-null   int64  
 12  no_of_style_change     1197 non-null   int64  
 13  no_of_workers          1197 non-null   float64
 14  actual_productivity    1197 non-null   float64
dtypes: f

In [4]:
import pandas as pd

# Cargar el archivo
df = pd.read_csv("garments_worker_productivity.csv")

# Eliminar la columna 'date'
df.drop(columns=['date'], inplace=True)

# Convertir 'actual_productivity' a 0 y 1 según la condición
df['actual_productivity'] = (df['actual_productivity'] >= 0.5).astype(int)


In [5]:
# Revisar cuántos valores faltantes hay por columna
print(df.isnull().sum())

# Imputar 'wip' con la media (puedes cambiar por mediana, 0, etc. según análisis)
df['wip'].fillna(df['wip'].mean(), inplace=True)


quarter                    0
department                 0
day                        0
team                       0
targeted_productivity      0
smv                        0
wip                      506
over_time                  0
incentive                  0
idle_time                  0
idle_men                   0
no_of_style_change         0
no_of_workers              0
actual_productivity        0
dtype: int64


<ipython-input-5-237570053>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['wip'].fillna(df['wip'].mean(), inplace=True)


In [6]:
from scipy import stats
import numpy as np

# Seleccionar columnas numéricas (excepto la variable objetivo)
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.drop('actual_productivity')

# Calcular z-scores
z_scores = np.abs(stats.zscore(df[numeric_cols]))

# Detectar outliers univariados (z > 3)
outliers_univariados = (z_scores > 3)

# Cuántos outliers hay por columna
outliers_por_columna = outliers_univariados.sum(axis=0)
print(pd.Series(outliers_por_columna, index=numeric_cols))


team                      0
targeted_productivity    30
smv                      10
wip                       8
over_time                 5
incentive                10
idle_time                 4
idle_men                 15
no_of_style_change       33
no_of_workers             0
dtype: int64


In [7]:
# Convertir columnas categóricas en variables dummy
categorical_cols = df.select_dtypes(include='object').columns
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


In [8]:
from sklearn.preprocessing import StandardScaler

# Separar variables predictoras (X) y objetivo (y)
X = df_encoded.drop('actual_productivity', axis=1)
y = df_encoded['actual_productivity']

# Escalar variables predictoras
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convertir a DataFrame para mantener nombres de columnas
X_preprocessed = pd.DataFrame(X_scaled, columns=X.columns)


In [10]:
from sklearn.model_selection import train_test_split

# División 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X_preprocessed, y, test_size=0.2, random_state=42, stratify=y
)


In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Diccionario para guardar resultados
modelos = {}

# k-NN
modelos['KNN'] = KNeighborsClassifier(n_neighbors=5)
# SVM
modelos['SVM'] = SVC(kernel='rbf', C=1.0)
# Regresión logística
modelos['LogReg'] = LogisticRegression(max_iter=1000)
# Árbol de decisión
modelos['DecisionTree'] = DecisionTreeClassifier(max_depth=5)
# Random Forest
modelos['RandomForest'] = RandomForestClassifier(n_estimators=100, max_depth=5)
# Naive Bayes
modelos['NaiveBayes'] = GaussianNB()

# Entrenamiento y evaluación
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{nombre} Accuracy: {acc:.4f}")


KNN Accuracy: 0.8917
SVM Accuracy: 0.8958
LogReg Accuracy: 0.8792
DecisionTree Accuracy: 0.9125
RandomForest Accuracy: 0.8875
NaiveBayes Accuracy: 0.8625
